# Tool

To ensure the smooth execution of function calls within the `Session` object, it is essential to register tools through the `Tool` object. This approach facilitates effective management and application of tools.

In [1]:
from lionagi import Tool

## Tool Construction

If you already have a well structured schema, you may directly construct a `Tool` object using the function and the corresponding schema.

In [2]:
tool_1=[
    {
        "type": "function",
        "function": {
            "name": "multiply",
            "description": "Perform multiplication on two numbers",
            "parameters": {
                "type": "object",
                "properties": {
                    "number1": {
                        "type": "number",
                        "description": "a number to multiply, e.g. 5.34",
                    },
                    "number2": {
                        "type": "number",
                        "description": "a number to multiply, e.g. 17",
                    },
                },
                # specify which parameters are required for the model to respond when function calling
                "required": ["number1", "number2"],
            },
        }
    }
]

def multiply(number1, number2):
    return number1*number2

In [3]:
tool_mul = Tool(func=multiply, schema_=tool_1[0])

If you do not want to be bothered by writing a schema, `func_to_tool` can help you generate a schema and construct a `Tool` object for you. 

In the generated schema, parameter types are inferred from the function type hints, and descriptions are based on the docstring.

Since the schema is crucial for function calling, a well-structured docstring is essential for the quality of the schema constructed for you. We currently support Google and reST style docstrings.

In [4]:
from lionagi import func_to_tool

In [5]:
# No docstring and type hints
def add_v1(number1, number2):
    return number1+number2

tool_add_v1 = func_to_tool(add_v1)
tool_add_v1.schema_

{'type': 'function',
 'function': {'name': 'add_v1',
  'description': 'No description available.',
  'parameters': {'type': 'object',
   'properties': {'number1': {'type': 'string',
     'description': 'No description available.'},
    'number2': {'type': 'string', 'description': 'No description available.'}},
   'required': ['number1', 'number2']}}}

In [6]:
# google style docstring (default)
def add_v2(number1:float, number2:float):
    '''
    Add up two numbers.

    Args:
        number1: First number to add.
        number2: Second number to add.
    
    Returns:
        The sum of number1 and number2.

    '''
    return number1+number2

tool_add_v2 = func_to_tool(add_v2)
tool_add_v2.schema_

{'type': 'function',
 'function': {'name': 'add_v2',
  'description': 'Add up two numbers.',
  'parameters': {'type': 'object',
   'properties': {'number1': {'type': 'number',
     'description': 'First number to add.'},
    'number2': {'type': 'number', 'description': 'Second number to add.'}},
   'required': ['number1', 'number2']}}}

In [7]:
# reST style docstring
def add_v3(number1:float, number2:float):
    '''
    Add up two numbers.

    :param number1: First number to add.
    :param number2: Second number to add.
    :returns:  The sum of number1 and number2.
    '''
    return number1+number2

tool_add_v3 = func_to_tool(add_v3, docstring_style='reST')
tool_add_v3.schema_

{'type': 'function',
 'function': {'name': 'add_v3',
  'description': 'Add up two numbers.',
  'parameters': {'type': 'object',
   'properties': {'number1': {'type': 'number',
     'description': 'First number to add.'},
    'number2': {'type': 'number', 'description': 'Second number to add.'}},
   'required': ['number1', 'number2']}}}

## Tools in Session

It is crucial to register all tools needed for each session before using them. In the following steps, you can specify which tool or set of tools you want to use in that step.

In [8]:
import lionagi as li

In [9]:
# and now some messages
system = "you are asked to perform as a function picker and parameter provider"
task = "Think step by step, understand the following basic math question and provide parameters for function calling."

# when using respond_mode as json to enforce output format, you need to provide specifying details in instruction
json_format = {"number1": "x", "number2": "y"}

instruct1 = {"Task": task, "json_format": json_format}

In [10]:
question1 = "There are [basketball, football, backpack, water bottle, strawberry, tennis ball, rockets]. each comes in four different colors, what is the number of unique kinds of ball?"
question2 = "There are three fruits in total, each with 2 different colors, how many unique kinds of fruits are there?"
question3 = "There are three cards facing up and two cards facing down. How many cards are there in total?"

In [11]:
session = li.Session(system=system)

session.register_tools([tool_mul, tool_add_v3])
session.llmconfig.update({
    "temperature":0.35,
    "tool_choice": "auto", 
    "response_format": {'type':'json_object'}
})

If you want to specify a single tool to be used in this step, you can pass in:

- the name of the tool (str)
- the `Tool` object
- a tool schema

If you want to specify a subset of tools, you can pass in a list containing any of these three types.

In [12]:
response1 = await session.initiate(instruction=instruct1, context=question1, tools='multiply')

In [13]:
response2 = await session.followup(instruction=instruct1, context=question2, tools=tool_1[0])

In [14]:
response3 = await session.followup(instruction=instruct1, context=question3, tools=tool_add_v3)

In [15]:
li.lcall(session.conversation.messages, lambda x: print(f'{x} \n'));

Message(role=system, name=system, content='{'system': 'you are asked to perform as a function picker and parameter provider'}') 

Message(role=user, name=user, content='{'instruction': {'Task': 'Think step by step, understand the following basic math question and provide parameters for function calling.', 'json_format': {'number1': 'x', 'number2': 'y'}}, 'context': 'There are [basketball, football, backpack, water bottle, strawberry, tennis ball, rockets]. each comes in four different colors, what is the number of unique kinds of ball?'}') 

Message(role=assistant, name=assistant, content='{
  "tool_uses": [
    {
      "recipient_name": "functions.multiply",
    ...') 

Message(role=assistant, name=func_call, content='{'function': 'multiply', 'arguments': {'number1': 3, 'number2': 4}, 'output': 12}') 

Message(role=user, name=user, content='{'instruction': {'Task': 'Think step by step, understand the following basic math question and provide parameters for function calling.', 'json_fo